In [30]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [31]:
X = cv2.imread('./images/simple_binary.png')

In [32]:
X = cv2.cvtColor(X,cv2.COLOR_BGR2GRAY)
X = X[0:234][0:234]

In [33]:
p_size = 80
patches_x = 3
patches_y = 3

In [34]:
X.shape

(234, 236)

In [37]:
offset = 1
for i in range(0, X.shape[0], p_size):
    for j in range(0, X.shape[1], p_size):
        s = np.zeros(X.shape)
        s[i+1:i+p_size, j+1:j+p_size] = np.array(X[i+1:i+p_size, j+1:j+p_size])
        cv2.imwrite(f"./image{offset}.png", s)
        offset += 1

True